In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [2]:
# Stap 2: Inlezen en voorbereiden van de data
# Lees de CSV-bestand in (vervang 'path/to/your/dataset.csv' met de daadwerkelijke pad naar je CSV)
df = pd.read_csv('cleaned_data.csv', sep=';')

# Verken de data (optioneel)
print(df.head())  # Print de eerste paar regels van de dataset
print(df.info())  # Geef een overzicht van de dataset

# Selecteer de features (X) en de target variabele (y)
X = df.drop(['HeartDisease' ,'CaseNumber', 'LastName', 'PostCode'], axis=1)  # Verwijder niet-relevante kolommen
y = df['HeartDisease']

# Converteer categorische kolommen naar numeriek (One-Hot Encoding voor categorische features)
X = pd.get_dummies(X, drop_first=True)

# Splits de data in een train- en testset (80% trainen, 20% testen)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    CaseNumber  LastName  PostCode  Age Sex ChestPainType  RestingBP  \
0  825-37-0603     Stout     82490   46   M           ASY        100   
1  533-80-6803  Gonzalez       696   56   M           NAP        125   
2  161-71-2407     Smith     44762   42   M           ASY        105   
3  598-01-6389  Townsend     21753   38   M           NAP        100   
4  142-50-9256      Ford     74606   56   M           ASY        115   

   Cholesterol  FastingBS  RestingECG  MaxHR ExerciseAngina  Oldpeak ST_Slope  \
0          244          1           1    133              N       26     Flat   
1          244          1           0     98              N       20     Flat   
2          244          1           0    128              Y       15     Down   
3          244          0           0    179              N       11       Up   
4          244          1           1     82              N       10       Up   

   HeartDisease  
0             1  
1             1  
2             1  
3       

In [3]:
# Stap 3: Model Selectie en Training
# Definieer de modellen die je wilt proberen
models = {
    "Linear Regression": LinearRegression(),  # Voorbeeldmodel, gebruik voor vergelijking, niet echt voor classificatie
    "Logistic Regression": LogisticRegression(max_iter=10000),
    "Ridge Classifier": RidgeClassifier(),
    "Lasso Regression": Lasso(),  # Voorbeeldmodel, gebruik voor vergelijking, niet echt voor classificatie
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
}

# Train elk model en evalueer de prestaties
results = {}
for name, model in models.items():
    try:
        model.fit(X_train, y_train)  # Train het model
        
        # Check of predict_proba beschikbaar is, anders gebruik predict direct voor ROC AUC
        if hasattr(model, "predict_proba"):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            # Voor lineaire regressie en lasso, cast naar binair resultaat
            y_pred_proba = model.predict(X_test)
            y_pred_proba = (y_pred_proba >= 0.5).astype(int)  # Simuleer classificatie output
        
        y_pred = model.predict(X_test)  # Voorspel met de testdata
        accuracy = accuracy_score(y_test, y_pred)  # Bereken de nauwkeurigheid
        roc_auc = roc_auc_score(y_test, y_pred_proba)  # Bereken de ROC AUC
        f1 = f1_score(y_test, y_pred)  # Bereken de F1-score
        
        results[name] = {'Accuracy': accuracy, 'ROC AUC': roc_auc, 'F1 Score': f1}
        # Print de evaluatie metrics
        print(f"{name} - Accuracy: {accuracy:.4f}, ROC AUC: {roc_auc:.4f}, F1 Score: {f1:.4f}")
    
    except Exception as e:
        print(f"Error training {name}: {e}")

# Bepaal welk model de beste prestaties levert op basis van ROC AUC
best_model = max(results, key=lambda x: results[x]['ROC AUC'])
print(f"\nBest model based on ROC AUC: {best_model}")

Error training Linear Regression: Classification metrics can't handle a mix of binary and continuous targets
Logistic Regression - Accuracy: 0.8634, ROC AUC: 0.9340, F1 Score: 0.8768
Ridge Classifier - Accuracy: 0.8634, ROC AUC: 0.8616, F1 Score: 0.8792
Error training Lasso Regression: Classification metrics can't handle a mix of binary and continuous targets
Decision Tree - Accuracy: 0.7596, ROC AUC: 0.7504, F1 Score: 0.7944
Random Forest - Accuracy: 0.8962, ROC AUC: 0.9506, F1 Score: 0.9100
Gradient Boosting - Accuracy: 0.8907, ROC AUC: 0.9383, F1 Score: 0.9038

Best model based on ROC AUC: Random Forest
